# SteadierCom simulations

In [1]:
import pandas as pd
import reframed
import os
import json
import numpy as np
from matplotlib import pyplot as plt
import re
import sys
sys.path.append('../functions')

import translation_dicts
import general_functions as general_func
import colors_community as color_func

import steadiercom_data_processing as steadier_data_func

from steadiercom import steadiercom

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
compounds_dict, source_dict,substrate_dict, gas_sheet_dict, community_dict = translation_dicts.translation_dicts()

### Load data

In [4]:
all_mags_paper = general_func.read_allmags_data()

with open('../output/community_production.json') as f:
    community_production = json.load(f)

top_99 = pd.read_csv("../output/steadiercom_sample_0.1.3/communities_top99.tsv",sep="\t",header=None)
top_99.set_index(1,inplace=True)
top_99.index.name="MAG"
top_99.columns = ["community"]

media_db = pd.read_csv("../output/steadiercom_sample_0.1.3/media_db.tsv",sep="\t")
media_db.set_index("medium",inplace=True)

met_chebi_class = pd.read_csv("../output/met_chebi_class.tsv",index_col=0,sep="\t")

**Define interesting classes**

In [5]:
interesting_super_classes = list(met_chebi_class["self defined super class"].unique())
interesting_super_classes.remove("other")
interesting_super_classes.remove("aromatic compounds")
interesting_super_classes.remove("organophosphorus compounds")
interesting_super_classes.remove("organosulfur compounds")
interesting_super_classes.remove("organonitrogen compounds")
interesting_super_classes.remove("inorganic ions and atoms")

chebi_interesting = met_chebi_class[met_chebi_class["self defined super class"].isin(interesting_super_classes)]

### Load models

**Load universal model**

In [6]:
model_uni = reframed.load_cbmodel("../input/universe_bacteria.xml")

**Load other models**

In [7]:
GEMs_dict = {}

directory = os.fsencode("../output/GEMs/GEMs_final/")

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print(filename)
    if filename.endswith(".xml"):
        GEMs_dict[filename[:-4]]= reframed.load_cbmodel("../output/GEMs/GEMs_final/"+filename,flavor="bigg")


CH7-bin.18.xml
CH15-bin.7.xml
CH13-bin.0.xml
CH1-bin.4.xml
CH13-bin.1.xml
CH13-bin.11.xml
CH15-bin.6.xml
CH7-bin.2.xml
CH13-bin.13.xml
CH1-bin.10.xml
CH1-bin.6.xml
CH13-bin.2.xml
CH13-bin.12.xml
CH15-bin.5.xml
CH7-bin.1.xml
CH15-bin.1.xml
CH1-bin.3.xml
CH13-bin.17.xml
CH15-bin.0.xml
CH7-bin.4.xml
CH7-bin.23.xml
CH7-bin.6.xml
CH15-bin.22.xml
CH15-bin.2.xml
CH1-bin.1.xml
CH13-bin.4.xml
CH13-bin.14.xml
CH15-bin.23.xml
CH7-bin.20.xml
CH8-bin.8.xml
CH8-bin.9.xml
CH8-bin.25.xml
CH14-bin.4.xml
CH14-bin.1.xml
CH8-bin.21.xml
CH14-bin.2.xml
CH8-bin.22.xml
CH8-bin.2.xml
CH8-bin.29.xml
CH8-bin.7.xml
CH8-bin.6.xml
CH3-bin.2.xml
CH8-bin.14.xml
CH8-bin.16.xml
CH3-bin.0.xml
CH8-bin.5.xml
CH3-bin.1.xml
CH8-bin.17.xml
CH7-bin.11.xml
CH9-bin.1.xml
CH15-bin.12.xml
CH13-bin.25.xml
CH15-bin.13.xml
CH9-bin.0.xml
CH7-bin.12.xml
CH9-bin.2.xml
CH7-bin.9.xml
CH15-bin.10.xml
CH7-bin.8.xml
CH7-bin.13.xml
CH7-bin.17.xml
CH15-bin.8.xml
CH15-bin.15.xml
CH7-bin.16.xml
CH9-bin.6.xml
CH9-bin.4.xml
CH15-bin.17.xml
CH1-bi

for MAG, model in GEMs_dict.items():
    
    
    if "R_EX_2h3mb_e" in model.reactions:
        print(True)                     
        rxns = model.get_metabolite_reactions("M_2h3mb_e")
        
        model.remove_reactions(rxns)
        model.remove_metabolite("M_2h3mb_e")
        model.update()
        GEMs_dict[MAG]=model
        
        

In [8]:
community_GEMs = {}
for community,MAGs in top_99.groupby("community").groups.items():
    
    community_GEMs[community]=[]
    for MAG in MAGs:
        reframed.Environment.empty(GEMs_dict[MAG],inplace=True)
        community_GEMs[community].append(GEMs_dict[MAG])

### Process other data

In [9]:
medias = {}

for media in set(media_db["compound"].index):
    
    medias[media] = media_db[media_db.index==media].compound.values


**Ensure that carbon is limiting**

In [99]:
community_media = {}

for community_id in community_GEMs.keys():
    
    if community_id in ["CM_A","CM_P","M_A","M_P"]:
        community_media[community_id] = reframed.Environment.from_compounds(medias["SC2_C"])
        #community_media[community_id].R_EX_glc__D_e = (-1/10,0)
        community_media[community_id].R_EX_cellb_e = (-1/20,0)
        community_media[community_id].R_EX_cell3_e = (-1/30,0)
        community_media[community_id].R_EX_cell4_e = (-1/40,0)
        community_media[community_id].R_EX_cell5_e = (-1/50,0)
        community_media[community_id].R_EX_ascb__L_e = (-1e-2,0)
        community_media[community_id].R_EX_pydxn_e = (-1e-2,0)
       
    
    elif community_id.split("_")[1]=="X":
        community_media[community_id] = reframed.Environment.from_compounds(medias["SC1_X"])
        community_media[community_id].R_EX_xylb_e=(-(6*4)/(100*3),0)
        community_media[community_id].R_EX_xylan4_e=(-(6*4)/(260*3),0)
        community_media[community_id].R_EX_xylan8_e=(-(6*4)/(520*3),0)
        community_media[community_id].R_EX_ascb__L_e = (-0.00001,0)
        community_media[community_id].R_EX_pydxn_e = (-0.00001,0)
        

    elif (community_id.split("_")[1]=="P") or (community_id.split("_")[1]=="A"):
        community_media[community_id] = reframed.Environment.from_compounds(medias["SC1_C"])
        community_media[community_id].R_EX_cellb_e = (-1/20,0)
        community_media[community_id].R_EX_cell3_e = (-1/30,0)
        community_media[community_id].R_EX_cell4_e = (-1/40,0)
        community_media[community_id].R_EX_cell5_e = (-1/50,0)
        community_media[community_id].R_EX_ascb__L_e = (-1e-2,0)
        community_media[community_id].R_EX_pydxn_e = (-1e-2,0)

       
    else:
        continue

    community_media[community_id].R_EX_so4_e = (-10000,0)
    community_media[community_id].R_EX_cys__L_e = (-1e-2,1000)

In [101]:
relative_abundance_estimate = {}

for MAG in GEMs_dict.keys():
    relative_abundance_estimate["_".join(re.split("[-.]",MAG))] = all_mags_paper.loc[MAG,"new_coverage"]/100


### SteadierCom

- w_R = protein fraction allocated to ribosomal proteins per unit of growth rate. 
    - Higher w_R means lower translation -> slower metabolism?
- w_E: w_i is the proteome fraction to be invested in a enzyme i. The sum of w_i across all enzymes is w_e.
    - Higher w_E means higher cost for enzymes.

In [69]:
community_id="CD_P"
estimate_abundance = {model.id:relative_abundance_estimate[model.id] for model in community_GEMs[community_id]}


community_model = reframed.Community(community_id,community_GEMs[community_id])
community_media[community_id].apply(community_model.merged_model,inplace=True,exclusive=True,warning=False)
sol = steadiercom.SteadierCom(community_model,abundance=estimate_abundance,growth=None,allocation=True,w_e=0.002,w_r=0.2) #growth=0.0089
        

In [70]:
sol

Community growth: 0.017263912789903815
CH15_bin_0	0.2667543717083474
CH15_bin_23	0.11509113120527982
CH15_bin_1	0.10696892639962761
CH15_bin_7	0.08762248391140315
CH15_bin_8	0.06349380899982422
CH15_bin_22	0.06152125076429115
CH15_bin_6	0.05050354746754619
CH15_bin_17	0.049223883641150296
CH15_bin_16	0.04743490929587535
CH15_bin_13	0.03720427115737394
CH15_bin_12	0.028447571412941675
CH15_bin_5	0.02663830418572999
CH15_bin_10	0.024861490402371858
CH15_bin_15	0.019432974968019722
CH15_bin_2	0.014801074480217587

In [71]:
sol_df = pd.DataFrame(sol.cross_feeding())
sol_df.columns=["donor","receiver","compound","rate","mass_rate"]
sol_df

,donor,receiver,compound,rate,mass_rate
0,CH15_bin_0,None,M_ac_e,0.005727,0.000338
1,CH15_bin_23,None,M_ac_e,0.050080,0.002957
2,CH15_bin_1,None,M_ac_e,0.126410,0.007464
3,CH15_bin_7,None,M_ac_e,0.115026,0.006792
4,CH15_bin_8,None,M_ac_e,0.041205,0.002433
...,...,...,...,...,...
620,CH15_bin_8,CH15_bin_17,M_lac__L_e,0.007778,0.000693
621,CH15_bin_8,CH15_bin_16,M_lac__L_e,0.033761,0.003007
622,CH15_bin_8,CH15_bin_15,M_lac__L_e,0.008797,0.000784
623,CH15_bin_5,CH15_bin_12,M_hxan_e,0.000100,0.000014


In [72]:
sol_df[(sol_df.donor.isna())].sort_values("mass_rate").tail(15)

,donor,receiver,compound,rate,mass_rate
47,None,CH15_bin_8,M_ascb__L_e,0.007225,0.001273
71,None,CH15_bin_1,M_cellb_e,0.004255,0.001456
75,None,CH15_bin_15,M_cellb_e,0.005474,0.001874
572,None,CH15_bin_7,M_cell3_e,0.003732,0.001883
511,None,CH15_bin_22,M_so4_e,0.023460,0.002254
65,None,CH15_bin_0,M_cell4_e,0.003610,0.002406
73,None,CH15_bin_5,M_cellb_e,0.007865,0.002692
67,None,CH15_bin_6,M_cell4_e,0.004749,0.003166
70,None,CH15_bin_0,M_cellb_e,0.015303,0.005238
72,None,CH15_bin_16,M_cellb_e,0.015682,0.005368


In [73]:
community_id="CD_A"
estimate_abundance = {model.id:relative_abundance_estimate[model.id] for model in community_GEMs[community_id]}


community_model = reframed.Community(community_id,community_GEMs[community_id])
community_media[community_id].apply(community_model.merged_model,inplace=True,exclusive=True,warning=False)
sol = steadiercom.SteadierCom(community_model,abundance=estimate_abundance,growth=None,allocation=True,w_e=0.002,w_r=0.2) #growth=0.0089
        

In [74]:
sol

Community growth: 0.013246161909869146
CH13_bin_12	0.4260844750672784
CH13_bin_11	0.11846751264866046
CH13_bin_4	0.11716312990104798
CH13_bin_14	0.11231778309898487
CH13_bin_25	0.07807152551650923
CH13_bin_17	0.055579155126144586
CH13_bin_0	0.05196867570675144
CH13_bin_1	0.015202548261359689
CH13_bin_13	0.01325076496443104
CH13_bin_2	0.011894429708832362

In [75]:
sol_df = pd.DataFrame(sol.cross_feeding())
sol_df.columns=["donor","receiver","compound","rate","mass_rate"]
sol_df

,donor,receiver,compound,rate,mass_rate
0,CH13_bin_12,None,M_ac_e,0.375886,0.022194
1,CH13_bin_11,None,M_ac_e,0.025101,0.001482
2,CH13_bin_4,None,M_ac_e,0.062949,0.003717
3,CH13_bin_14,None,M_ac_e,0.106483,0.006287
4,CH13_bin_25,None,M_ac_e,0.086412,0.005102
...,...,...,...,...,...
351,None,CH13_bin_14,M_cell3_e,0.011221,0.005660
352,None,CH13_bin_25,M_cell3_e,0.003876,0.001955
353,None,CH13_bin_17,M_cell3_e,0.009843,0.004965
354,None,CH13_bin_0,M_cell3_e,0.006476,0.003267


In [76]:
sol_df[(sol_df.donor.isna())].sort_values("mass_rate").tail(15)

,donor,receiver,compound,rate,mass_rate
87,None,CH13_bin_4,M_cys__L_e,0.008908,0.001079
48,None,CH13_bin_2,M_cellb_e,0.004299,0.001472
306,None,CH13_bin_2,M_ascb__L_e,0.009216,0.001623
335,None,CH13_bin_11,M_pydxn_e,0.010000,0.001692
352,None,CH13_bin_25,M_cell3_e,0.003876,0.001955
272,None,CH13_bin_25,M_so4_e,0.021321,0.002048
43,None,CH13_bin_12,M_cellb_e,0.009085,0.003110
354,None,CH13_bin_0,M_cell3_e,0.006476,0.003267
353,None,CH13_bin_17,M_cell3_e,0.009843,0.004965
351,None,CH13_bin_14,M_cell3_e,0.011221,0.005660


In [77]:
community_id="CM_A"
estimate_abundance = {model.id:relative_abundance_estimate[model.id] for model in community_GEMs[community_id]}


community_model = reframed.Community(community_id,community_GEMs[community_id])
community_media[community_id].apply(community_model.merged_model,inplace=True,exclusive=True,warning=False)
sol = steadiercom.SteadierCom(community_model,abundance=estimate_abundance,growth=None,allocation=True,w_e=0.002,w_r=0.2) #growth=0.0089
        

In [78]:
sol

Community growth: 0.011431978985001373
CH8_bin_22	0.512008796765076
CH8_bin_17	0.0936782313071004
CH8_bin_7	0.08529650163350129
CH8_bin_2	0.06456309050239091
CH8_bin_6	0.0497924060227011
CH8_bin_25	0.04972226797089937
CH8_bin_5	0.03448615673630647
CH8_bin_9	0.024486104882945666
CH8_bin_16	0.020965000439866772
CH8_bin_21	0.02053849306557408
CH8_bin_8	0.018866748861973016
CH8_bin_29	0.012999032379377573
CH8_bin_14	0.012597169432287298

In [79]:
sol_df = pd.DataFrame(sol.cross_feeding())
sol_df.columns=["donor","receiver","compound","rate","mass_rate"]
sol_df

,donor,receiver,compound,rate,mass_rate
0,CH8_bin_22,None,M_4hba_e,0.000001,1.620334e-07
1,CH8_bin_22,None,M_ac_e,0.395078,2.332693e-02
2,CH8_bin_17,None,M_ac_e,0.100706,5.946046e-03
3,CH8_bin_2,None,M_ac_e,0.060349,3.563269e-03
4,CH8_bin_6,None,M_ac_e,0.000306,1.804580e-05
...,...,...,...,...,...
495,CH8_bin_2,None,M_h2_e,0.005317,1.071848e-05
496,CH8_bin_25,None,M_h2_e,0.029991,6.045486e-05
497,CH8_bin_8,None,M_h2_e,0.021262,4.285966e-05
498,CH8_bin_5,CH8_bin_8,M_mal__L_e,0.003108,4.104383e-04


In [80]:
sol_df[(sol_df.donor.isna())].sort_values("mass_rate").tail(15)

,donor,receiver,compound,rate,mass_rate
43,None,CH8_bin_21,M_cellb_e,0.003134,0.001073
389,None,CH8_bin_2,M_so4_e,0.013499,0.001297
394,None,CH8_bin_16,M_so4_e,0.014024,0.001347
494,None,CH8_bin_8,M_ascb__L_e,0.007728,0.001361
390,None,CH8_bin_6,M_so4_e,0.016562,0.001591
489,None,CH8_bin_9,M_cell3_e,0.003386,0.001708
387,None,CH8_bin_17,M_so4_e,0.023909,0.002297
42,None,CH8_bin_16,M_cellb_e,0.007554,0.002586
487,None,CH8_bin_6,M_cell3_e,0.005671,0.002860
39,None,CH8_bin_7,M_cellb_e,0.011475,0.003928


In [81]:
community_id="M_P"
estimate_abundance = {model.id:relative_abundance_estimate[model.id] for model in community_GEMs[community_id]}


community_model = reframed.Community(community_id,community_GEMs[community_id])
community_media[community_id].apply(community_model.merged_model,inplace=True,exclusive=True,warning=False)
sol = steadiercom.SteadierCom(community_model,abundance=estimate_abundance,growth=None,allocation=True,w_e=0.002,w_r=0.2) #growth=0.0089
        

In [82]:
sol

Community growth: 0.01823226551396384
CH1_bin_6	0.47867017091688635
CH1_bin_9	0.32978563394210214
CH1_bin_3	0.06561681567811156
CH1_bin_4	0.04906440588016862
CH1_bin_8	0.04096116766082448
CH1_bin_1	0.021679091461244533
CH1_bin_10	0.014222714460662396

In [83]:
sol_df = pd.DataFrame(sol.cross_feeding())
sol_df.columns=["donor","receiver","compound","rate","mass_rate"]
sol_df

,donor,receiver,compound,rate,mass_rate
0,CH1_bin_6,None,M_ac_e,3.430155e-03,2.025297e-04
1,CH1_bin_9,None,M_ac_e,4.439434e-01,2.621215e-02
2,CH1_bin_3,None,M_ac_e,9.221700e-02,5.444851e-03
3,CH1_bin_4,None,M_ac_e,1.204314e-02,7.110739e-04
4,CH1_bin_1,None,M_ac_e,2.856147e-02,1.686380e-03
...,...,...,...,...,...
216,None,CH1_bin_3,M_indole_e,8.528535e-21,9.990975e-22
217,None,CH1_bin_1,M_indole_e,2.817737e-21,3.300911e-22
218,None,CH1_bin_9,M_cell5_e,1.084595e-02,8.988212e-03
219,None,CH1_bin_8,M_cell5_e,9.154053e-03,7.586112e-03


In [84]:
sol_df[(sol_df.donor.isna())].sort_values("mass_rate").tail(15)

,donor,receiver,compound,rate,mass_rate
167,None,CH1_bin_6,M_pi_e,0.008606,0.000826
164,None,CH1_bin_8,M_nh4_e,0.048679,0.000878
185,None,CH1_bin_4,M_so4_e,0.009387,0.000902
51,None,CH1_bin_3,M_cys__L_e,0.007626,0.000924
32,None,CH1_bin_3,M_cellb_e,0.002802,0.000959
160,None,CH1_bin_6,M_nh4_e,0.054763,0.000988
220,None,CH1_bin_8,M_pydxn_e,0.010000,0.001692
18,None,CH1_bin_10,M_ascb__L_e,0.010000,0.001761
30,None,CH1_bin_10,M_cell4_e,0.002942,0.001961
219,None,CH1_bin_8,M_cell5_e,0.009154,0.007586


In [94]:
community_media["CM_P"]

R_EX_zn2_e	-10.0	inf
R_EX_mn2_e	-10.0	inf
R_EX_fe3_e	-10.0	inf
R_EX_k_e	-10.0	inf
R_EX_slnt_e	-10.0	inf
R_EX_so4_e	-10000	0
R_EX_ribflv_e	-10.0	inf
R_EX_cl_e	-10.0	inf
R_EX_nh4_e	-10.0	inf
R_EX_btn_e	-10.0	inf
R_EX_pydxn_e	-0.01	0
R_EX_ni2_e	-10.0	inf
R_EX_h2o_e	-10.0	inf
R_EX_pi_e	-10.0	inf
R_EX_fe2_e	-10.0	inf
R_EX_fol_e	-10.0	inf
R_EX_pnto__R_e	-10.0	inf
R_EX_cu2_e	-10.0	inf
R_EX_na1_e	-10.0	inf
R_EX_mobd_e	-10.0	inf
R_EX_cys__L_e	-0.01	1000
R_EX_mg2_e	-10.0	inf
R_EX_4abz_e	-10.0	inf
R_EX_h_e	-10.0	inf
R_EX_nac_e	-10.0	inf
R_EX_ca2_e	-10.0	inf
R_EX_cobalt2_e	-10.0	inf
R_EX_thm_e	-10.0	inf
R_EX_cellb_e	-0.05	0
R_EX_cell3_e	-0.03333333333333333	0
R_EX_cell4_e	-0.025	0
R_EX_cell5_e	-0.02	0
R_EX_ascb__L_e	-0.01	0

In [102]:
community_id="CD_X"
estimate_abundance = {model.id:relative_abundance_estimate[model.id] for model in community_GEMs[community_id]}


community_model = reframed.Community(community_id,community_GEMs[community_id])
community_media[community_id].apply(community_model.merged_model,inplace=True,exclusive=True,warning=False)
sol = steadiercom.SteadierCom(community_model,abundance=estimate_abundance,growth=None,allocation=True,w_e=0.002,w_r=0.2) #growth=0.0089
         

In [103]:
sol

Community growth: 0.00957149395779524
CH14_bin_1	0.7171884100081193
CH14_bin_2	0.24686756958480954
CH14_bin_4	0.03594402040707116

In [104]:
sol_df = pd.DataFrame(sol.cross_feeding())
sol_df.columns=["donor","receiver","compound","rate","mass_rate"]
sol_df

,donor,receiver,compound,rate,mass_rate
0,CH14_bin_1,None,M_ac_e,0.250550,1.479342e-02
1,CH14_bin_2,None,M_ac_e,0.089789,5.301495e-03
2,CH14_bin_4,None,M_ac_e,0.058100,3.430473e-03
3,CH14_bin_2,CH14_bin_1,M_acald_e,0.010156,4.474139e-04
4,CH14_bin_2,CH14_bin_4,M_acald_e,0.058053,2.557386e-03
...,...,...,...,...,...
69,None,CH14_bin_2,M_ascb__L_e,0.000010,1.761238e-06
70,CH14_bin_2,None,M_co_e,0.149162,4.178056e-03
71,None,CH14_bin_2,M_fol_e,0.000002,6.945621e-07
72,None,CH14_bin_2,M_ribflv_e,0.000001,3.987544e-07


In [105]:
sol_df[(sol_df.donor.isna())].sort_values("mass_rate").tail(15)

,donor,receiver,compound,rate,mass_rate
39,None,CH14_bin_2,M_k_e,0.000461,0.000018
53,None,CH14_bin_4,M_pi_e,0.000339,0.000033
38,None,CH14_bin_1,M_k_e,0.001340,0.000052
32,None,CH14_bin_2,M_h2o_e,0.003034,0.000055
19,None,CH14_bin_2,M_cys__L_e,0.000581,0.000070
52,None,CH14_bin_2,M_pi_e,0.002330,0.000224
50,None,CH14_bin_4,M_nh4_e,0.022954,0.000414
58,None,CH14_bin_4,M_so4_e,0.004457,0.000428
49,None,CH14_bin_2,M_nh4_e,0.034072,0.000615
51,None,CH14_bin_1,M_pi_e,0.006769,0.000650


In [106]:
community_id="CM_X"
estimate_abundance = {model.id:relative_abundance_estimate[model.id] for model in community_GEMs[community_id]}


community_model = reframed.Community(community_id,community_GEMs[community_id])
community_media[community_id].apply(community_model.merged_model,inplace=True,exclusive=True,warning=False)
sol = steadiercom.SteadierCom(community_model,abundance=estimate_abundance,growth=None,allocation=True,w_e=0.002,w_r=0.2) #growth=0.0089
         

In [107]:
sol

Community growth: 0.004258302470537485
CH9_bin_0	0.7877195926911693
CH9_bin_6	0.10803093842349695
CH9_bin_5	0.034865191786027
CH9_bin_1	0.025274297988987674
CH9_bin_2	0.022792898506106635
CH9_bin_4	0.021317080604212533

In [108]:
sol_df = pd.DataFrame(sol.cross_feeding())
sol_df.columns=["donor","receiver","compound","rate","mass_rate"]
sol_df

,donor,receiver,compound,rate,mass_rate
0,CH9_bin_0,CH9_bin_4,M_12ppd__R_e,0.002280,0.000173
1,CH9_bin_0,None,M_12ppd__R_e,0.052491,0.003994
2,CH9_bin_0,None,M_5drib_e,0.000422,0.000057
3,CH9_bin_0,None,M_ac_e,0.234183,0.013827
4,CH9_bin_6,None,M_ac_e,0.024132,0.001425
...,...,...,...,...,...
165,CH9_bin_2,CH9_bin_4,M_succ_e,0.000104,0.000012
166,CH9_bin_2,None,M_succ_e,0.010074,0.001169
167,CH9_bin_1,None,M_co_e,0.065744,0.001841
168,CH9_bin_4,CH9_bin_2,M_glyald_e,0.037057,0.003338


In [109]:
sol_df[(sol_df.donor.isna())].sort_values("mass_rate").tail(15)

,donor,receiver,compound,rate,mass_rate
73,None,CH9_bin_5,M_h2o_e,0.002123,0.000038
124,None,CH9_bin_6,M_pi_e,0.000454,0.000044
114,None,CH9_bin_5,M_nh4_e,0.003246,0.000059
113,None,CH9_bin_6,M_nh4_e,0.003628,0.000065
75,None,CH9_bin_2,M_h2o_e,0.004191,0.000076
115,None,CH9_bin_1,M_nh4_e,0.009000,0.000162
72,None,CH9_bin_6,M_h2o_e,0.010749,0.000194
123,None,CH9_bin_0,M_pi_e,0.003308,0.000317
125,None,CH9_bin_5,M_pi_e,0.003569,0.000343
116,None,CH9_bin_2,M_nh4_e,0.021716,0.000392


In [110]:
community_id="M_X"
estimate_abundance = {model.id:relative_abundance_estimate[model.id] for model in community_GEMs[community_id]}


community_model = reframed.Community(community_id,community_GEMs[community_id])
community_media[community_id].apply(community_model.merged_model,inplace=True,exclusive=True,warning=False)
sol = steadiercom.SteadierCom(community_model,abundance=estimate_abundance,growth=None,allocation=True,w_e=0.002,w_r=0.2) #growth=0.0089
         

In [111]:
sol

Community growth: 0.008848856814552348
CH3_bin_2	0.7637038106604236
CH3_bin_1	0.2205924833166971
CH3_bin_0	0.015703706022879464

In [114]:
sol_df = pd.DataFrame(sol.cross_feeding())
sol_df.columns=["donor","receiver","compound","rate","mass_rate"]

In [115]:
sol_df[(sol_df.donor.isna())].sort_values("mass_rate").tail(15)

,donor,receiver,compound,rate,mass_rate
42,None,CH3_bin_1,M_k_e,0.000381,0.000015
17,None,CH3_bin_1,M_cys__L_e,0.000180,0.000022
41,None,CH3_bin_2,M_k_e,0.001319,0.000052
54,None,CH3_bin_1,M_nh4_e,0.003365,0.000061
57,None,CH3_bin_1,M_pi_e,0.001925,0.000185
55,None,CH3_bin_0,M_nh4_e,0.014312,0.000258
56,None,CH3_bin_2,M_pi_e,0.006664,0.000640
32,None,CH3_bin_1,M_h2o_e,0.052657,0.000949
16,None,CH3_bin_2,M_cys__L_e,0.009807,0.001188
53,None,CH3_bin_2,M_nh4_e,0.071349,0.001287
